This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [30]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [28]:
# First, import the relevant modules
import requests
import numpy as np


In [29]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key=API_KEY')



In [30]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#Print the full JSON data object for the selected date
print(r.json())
print('\n')

#Print the data by calling the correct dictionary key. Used indexing to select unpack the nested list. 
print(r.json()['dataset_data']['data'][0])

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-01', 'end_date': '2017-01-02', 'frequency': 'daily', 'data': [['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Question 1

In [31]:
#Call the API using start/end dates for 2017
r_17 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=API_KEY')

#Check that the data and structure is identical to the practice example 
#print(r_17.json()['dataset_data']['data'])

### Question 2

In [32]:
#Convert JSON array using dict()
afx_dict = dict(r_17.json())

#Check data type to confirm
print(type(afx_dict))

<class 'dict'>


### Question 3

In [33]:
#Extract 'column_names' from JSON object to determine index of 'Open' data
column_names = afx_dict['dataset_data']['column_names']
#print(column_names)

#Open is the second item in each nested list within the 'data' dictionary

#Extract 'data' lists to iterate over
stock_data = afx_dict['dataset_data']['data']
#print(stock_data)

#Create empty list to store 'Open' data
opening_prices = []

#Iterate over stock_data and select 'Open' value from each. Do not append NoneType values. 
for i in stock_data:
    if i[1] != None:
        opening_prices.append(i[1])

#Select the max and min opening price values and print the result
print('The maximum opening price in 2017 was: $',max(opening_prices))
print('The minimum opening price in 2017 was: $',min(opening_prices))

The maximum opening price in 2017 was: $ 53.11
The minimum opening price in 2017 was: $ 34.0


### Question 4

In [34]:
#Create empty list
stock_data_dicts = []

#Create list of dictionaries matching column names to corresponding values
for day in stock_data:
    day_dict = dict(zip(column_names, day))
    stock_data_dicts.append(day_dict)
    
#Create empty list to store open/close differentials
daily_diff = []

#Subtract low price from high price for each day
for day in stock_data_dicts:
    if day['High'] != None and day['Low'] != None:
        daily_diff.append(day['High'] - day['Low'])

#Print the max value from the daily_diff list
print('The largest single-day change in price during 2017 was: $',round(max(daily_diff),2))
    

The largest single-day change in price during 2017 was: $ 2.81


### Question 5

In [35]:
#Create an empty list for the one-day closing price differences
one_day_diff = []

#Create a for loop with range() to subtract each day's closing price from that of the preceding day's. 
for idx,day in enumerate(stock_data_dicts):
    if idx != 0:
        one_day_diff.append(abs(stock_data_dicts[idx]['Close'] - stock_data_dicts[idx-1]['Close']))

#Print the result
print('The largest difference in closing price between two days in 2017 was: $',round(max(one_day_diff),2))

The largest difference in closing price between two days in 2017 was: $ 2.56


### Question 6

In [36]:
#Create an empty list for daily Trading Volume values
volume_list = []

#Append the Trading Volume value from each day to the volume_list
for i in stock_data_dicts:
    volume_list.append(i['Traded Volume'])

#Calculate the mean (sum divided by number of values) and print the result
print('The average daily trading volume in 2017 was: ',round(sum(volume_list)/len(volume_list),2))

The average daily trading volume in 2017 was:  89124.34


### Question 7

In [37]:
#Sort the volume list in ascending order (default)
volume_list = sorted(volume_list)

#Find the length of the list to determine the middle (median) value
l = len(volume_list)

#Calculate the middle length index (for odd number of values)
median_index = int((l+1)/2) - 1
#print(median_index)

#Use calculated index to extract median value
median = round(volume_list[median_index])

#Print the result
print('The median trading volume in 2017 was :',median)

The median trading volume in 2017 was : 76286
